<a href="https://colab.research.google.com/github/hemanthsunny/knowledge_graphs_0.1/blob/master/first_kg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import re
import pandas as pd
import bs4
import requests
import spacy
from spacy import displacy
nlp = spacy.load('en_core_web_sm')

In [0]:
from spacy.matcher import Matcher
from spacy.tokens import Span

In [0]:
import networkx as nx
import matplotlib.pyplot as plt
from tqdm import tqdm

In [0]:
pd.set_option('display.max_colwidth', 200)
%matplotlib inline

In [0]:
sentences = pd.read_csv('')